In [1]:
import numpy as np
from matplotlib import pyplot as plt

In [3]:
class BinaryClassifier:
    '''
        This is a logistic regression model
        ***Parameters***:
        -n_iterations: 1000,dtype=int
        -learning_rate: 0.01, dtype=float or int
        return: object
    '''
    def __init__(self, n_iterations = 1000, learning_rate = 0.01):
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.W = None
        self.biais = None
        self.costs = []

    def cost_function(self, y: list, y_predicted: list):
        '''
            y: list of the true targets
            y_predicted: list of predictions
            return: cost value dtype:float
        '''
        y = np.array(y)
        y = y.reshape(-1, 1)
        m = y.shape[0]
        epsilone = 10e-20 #To escape the calculation of log(0) 
        cost = -(1 / m) * np.sum(y * np.log(y_predicted + epsilone) + (1 - y) * np.log(1 - y_predicted + epsilone))
        return cost
        
    def sigmoide(self, z: list):
        '''
            return: np.ndarray
        '''
        return 1 / (1 + np.exp(-z))
        
    def fit(self, X: list, y: list):
        '''
            X: list of training features
            y: list of training targets
            note: y must have [0, 1] as unique value
        '''
        y = np.array(y)
        y = y.reshape(-1, 1)
        n_samples, n_features = X.shape
        self.W = np.zeros((n_features, 1))
        self.biais = 0

        for i in range(self.n_iterations):
            linear_model = np.dot(X, self.W) + self.biais
            y_predicted = self.sigmoide(linear_model)

            #Calcul of cost value
            cost = self.cost_function(y, y_predicted)
            self.costs.append(cost)

            #Calcul of gradient descents
            dW = (1 / n_samples) * np.dot(X.T, (y_predicted - y))
            db = (1 / n_samples) * np.sum(y_predicted - y)

            #Update the parameteters W and biais
            self.W = self.W - self.learning_rate * dW
            self.biais = self.biais - self.learning_rate * db
            

    def predict(self, X: list):
        '''
            X: list of testing features
            return: prediction list of X
        '''
        linear_model = np.dot(X, self.W) + self.biais
        y_predicted = self.sigmoide(linear_model)
        y_predicted = y_predicted > 0.5
        return (np.array(y_predicted, dtype='int')).flatten()

    def accuracy(self, y_true: list, y_pred: list):
        '''
            y_true: list of the true targets
            y_pred: list of the predicition targets
            
            note: y_true must have [0, 1] as unique value
            
            return: number of the correct prediction, dtype=float
        '''
        y_true = np.array(y_true)
        y_true = y_true.reshape(-1, 1)
        c = 0 
        for i in range(len(y_true)):
            if y_pred[i] == y_true[i]:
                c += 1
        acc = c/len(y_true)
        return acc

    def recall(self, y_true: list, y_pred: list):
        '''
            y_true: list of the true targets
            y_pred: list of the predicition targets
            
            note: y_true must have [0, 1] as unique value
            
            return: float
        '''
        y_pred = (np.array(y_pred)).flatten()
        tp = np.sum((y_true == 1) & (y_pred == 1))
        fp = np.sum((y_true == 1) & (y_pred == 0))
        return tp / (tp + fp) if (tp + fp) > 0 else 0

    def precision(self, y_true: list, y_pred: list):
        '''
            y_true: list of the true targets
            y_pred: list of the predicition targets

            note: y_true must have [0, 1] as unique value
            
            return: float
        '''
        y_pred = (np.array(y_pred)).flatten()
        tp = np.sum((y_true == 1) & (y_pred == 1))
        fp = np.sum((y_pred == 1) & (y_true == 0))
        return tp / (tp + fp) if (tp + fp) > 0 else 0
    
    def f1_score(self, y_true: list, y_pred: list):
        '''
            y_true: list of the true targets
            y_pred: list of the predicition targets
            
            note: y_true must have [0, 1] as unique value
            
            return: float
        '''
        p = self.precision(y_true, y_pred)
        r = self.recall(y_true, y_pred)
        return 2 * (p * r) / (p + r) if (p + r) > 0 else 0
        
    def score(self, y_true: list, y_pred: list):  
        '''
            y_true: list of the true targets
            y_pred: list of the predicition targets

            note: y_true must have [0, 1] as unique value
            
            return: show all metrics: accuracy, precision, recall, f1_score
        '''

        acc = self.accuracy(y_true, y_pred)
        print(f"Accuracy: {acc}")
        
        prec = self.precision(y_true, y_pred) 
        print(f"Precision: {prec}")
        
        rec = self.recall(y_true, y_pred)
        print(f"Recall: {rec}")
        
        f1 = self.f1_score(y_true, y_pred) 
        print(f"F1-Score: {f1}")

    def cost_plot(self):
        '''
            return: curve evaluation of cost value
        '''
        plt.figure()
        plt.plot(self.costs)
        plt.title('Cost value evaluation')
        plt.xlabel('iterations')
        plt.ylabel('cost')
        plt.show()